**Further train DeepCell Models with K's Data and Make Predictions**<br>
Starting from model trained with tn nuclear data, further train the model with resized K's images that is cut into 25 512x512 patches, one image at a time.
200 epochs, semanticdatagenerator, resized<br>

In [1]:
starting_model_path = 'tn1.0_nuclear_20221102.h5' # the model trained with nucleus data from the Tissuenet V1.0 dataset
epochs=200 # 20 sec per step 

In [6]:
import ssl 
ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
from sklearn.metrics.cluster import adjusted_rand_score

In [2]:
from tsp.AP import csi

import numpy as np
from skimage import io
from matplotlib import pyplot as plt
%matplotlib inline
from timeit import default_timer
import os
from skimage import img_as_ubyte

import deepcell
from deepcell import image_generators
from deepcell.utils.train_utils import rate_scheduler, get_callbacks, count_gpus
from deepcell_toolbox.utils import resize, tile_image, untile_image
from deepcell_toolbox.deep_watershed import deep_watershed
from deepcell.losses import weighted_categorical_crossentropy
from deepcell.model_zoo.panopticnet import PanopticNet

import tensorflow as tf
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.losses import MSE

print(tf.__version__)
print(deepcell.__version__)

2.7.1
0.11.1


In [7]:
#for ii in [1,2,3,4,5,6,7]:
ii=7
idx="training{}".format(ii)
INPUT_PATH="images/"+idx+"/"
model_name = 'tn1.0_nuclear_K_512x512resized_'+idx 
model_path = '{}.h5'.format(model_name)

import glob
FILENAMES = glob.glob(INPUT_PATH+"*_img.png")
print(len(FILENAMES))

imgs = [io.imread(CURR_IM_NAME)[:,:,0] for CURR_IM_NAME in FILENAMES]
X_train = tf.stack(imgs)
X_train = np.expand_dims(X_train, axis=-1)
print(X_train.shape)

masks = [io.imread(CURR_IM_NAME.replace("img","masks")) for CURR_IM_NAME in FILENAMES]
y_train = tf.stack(masks)
y_train = np.expand_dims(y_train, axis=-1)
print(y_train.shape)

# resize image
mpp_select=1.3 # based on DeepCell_tn_nuclear.ipynb, the best value is 1.2, but 1.3 makes scale factor an integer
shape = X_train.shape
scale_factor = mpp_select / 0.65
print(scale_factor)
new_shape = (int(shape[1] * scale_factor),
             int(shape[2] * scale_factor))

X_train_resized = resize(X_train, new_shape, data_format='channels_last')
y_train_resized = resize(y_train, new_shape, data_format='channels_last', labeled_image=True)
print(X_train_resized.shape)
print(y_train_resized.shape)

# cut each resized image into 5x5 patches of size 512x512 
print(X_train_resized.shape)
print(y_train_resized.shape)

x_starts=[0,400,400*2,400*3,X_train_resized.shape[1]-512]
x_ends=[x+512 for x in x_starts]
print(x_starts)
print(x_ends)

y_starts=[0,512,512*2,512*3,X_train_resized.shape[2]-512]
y_ends=[x+512 for x in y_starts]
print(y_starts)
print(y_ends)

sub_X=tf.concat([tf.concat([X_train_resized[:, x_starts[i]:x_ends[i], y_starts[j]:y_ends[j], :] for i in range(len(x_starts)) ], axis=0) for j in range(len(y_starts))], axis=0)
sub_X = np.array(sub_X, dtype=np.uint8) # tf.stack changes the dtype 
print(sub_X.shape)
sub_y=tf.concat([tf.concat([y_train_resized[:, x_starts[i]:x_ends[i], y_starts[j]:y_ends[j], :] for i in range(len(x_starts)) ], axis=0) for j in range(len(y_starts))], axis=0)
sub_y = np.array(sub_y, dtype=np.uint16) # tf.stack changes the dtype 
print(sub_y.shape)

# create dataloader
seed=0
min_objects = 5

transforms = ['inner-distance', 'outer-distance', 'fgbg']
transforms_kwargs = {'outer-distance': {'erosion_width': 0}}

# use augmentation for training but not validation
datagen = image_generators.SemanticDataGenerator(
    rotation_range=180,
    fill_mode='reflect',
    zoom_range=(0.75, 1.25),
    horizontal_flip=True,
    vertical_flip=True)

datagen_val = image_generators.CroppingDataGenerator()

batch_size = 4 # 8 causes memory outage

train_data = datagen.flow(
    {'X': sub_X, 'y': sub_y},
    seed=seed,
    transforms=transforms,
    transforms_kwargs=transforms_kwargs,
    min_objects=min_objects,
    batch_size=batch_size)

val_data = None

semantic_classes = [1, 1, 2] # inner distance, outer distance, fgbg

model = PanopticNet(
    backbone='resnet50',
    input_shape=(512,512,1),
    norm_method='whole_image',
    num_semantic_classes=semantic_classes)

lr = 1e-4
optimizer = Adam(lr=lr, clipnorm=0.001)
lr_sched = rate_scheduler(lr=lr, decay=0.99)

# Create a dictionary of losses for each semantic head

def semantic_loss(n_classes):
    def _semantic_loss(y_pred, y_true):
        if n_classes > 1:
            return 0.01 * weighted_categorical_crossentropy(
                y_pred, y_true, n_classes=n_classes)
        return MSE(y_pred, y_true)
    return _semantic_loss

loss = {}

# Give losses for all of the semantic heads
for layer in model.layers:
    if layer.name.startswith('semantic_'):
        n_classes = layer.output_shape[-1]
        loss[layer.name] = semantic_loss(n_classes)

model.compile(loss=loss, optimizer=optimizer)

model.load_weights(starting_model_path, by_name=True)

[(layer.name, layer.output_shape) for layer in filter(lambda x: x.name.startswith('semantic_'), model.layers)]


# fit the model
print('Training on', count_gpus(), 'GPUs.')

train_callbacks = get_callbacks(
    model_path,
    lr_sched=lr_sched,
#     monitor="val_loss",
    # monitor='loss', # training loss
    verbose=1)          

loss_history = model.fit(
    train_data,
    steps_per_epoch=train_data.y.shape[0] // batch_size, # 
    epochs=epochs, 
#     validation_data=val_data,
#     validation_steps=val_data.y.shape[0] // batch_size,
    callbacks=train_callbacks)

model.save_weights(model_path)

prediction_model = PanopticNet(
    backbone='resnet50',
    norm_method='whole_image',
    num_semantic_classes=[1, 1], # inner distance, outer distance
    input_shape= (512,512,1)
)

prediction_model.load_weights(model_path, by_name=True)

from deepcell.applications import NuclearSegmentation
app = NuclearSegmentation(prediction_model)
[(layer.name, layer.output_shape) for layer in filter(lambda x: x.name.startswith('semantic_'), app.model.layers)]




7
(7, 1040, 1159, 1)
(7, 1040, 1159, 1)
2.0
(7, 2080, 2318, 1)
(7, 2080, 2318, 1)
(7, 2080, 2318, 1)
(7, 2080, 2318, 1)
[0, 400, 800, 1200, 1568]
[512, 912, 1312, 1712, 2080]
[0, 512, 1024, 1536, 1806]
[512, 1024, 1536, 2048, 2318]
(175, 512, 512, 1)
(175, 512, 512, 1)
94781440/94765736 [==============================] - 1s 0us/step


/app/software/TensorFlow/2.7.1-foss-2021b-CUDA-11.4.1/lib/python3.9/site-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)
2023-07-03 00:31:29.624168: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /device:GPU:0 with 10429 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1080 Ti, pci bus id: 0000:5e:00.0, compute capability: 6.1


Training on 1 GPUs.
Epoch 1/200


2023-07-03 00:31:49.456756: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8202
2023-07-03 00:32:12.234251: W tensorflow/core/common_runtime/bfc_allocator.cc:275] Allocator (GPU_0_bfc) ran out of memory trying to allocate 536.16MiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.
2023-07-03 00:32:12.248984: W tensorflow/core/common_runtime/bfc_allocator.cc:275] Allocator (GPU_0_bfc) ran out of memory trying to allocate 528.00MiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.
2023-07-03 00:32:12.249013: W tensorflow/core/kernels/gpu_utils.cc:49] Failed to allocate memory for convolution redzone checking; skipping this check. This is benign and only means that we won't check cudnn for out-of-bounds reads and writes. This message will only be printed once.

42/42 [==============================] - 80s 649ms/step - loss: 0.0167 - semantic_0_loss: 0.0042 - semantic_1_loss: 0.0103 - semantic_2_loss: 0.0021 - lr: 1.0000e-04
Epoch 2/200
42/42 [==============================] - 23s 545ms/step - loss: 0.0146 - semantic_0_loss: 0.0037 - semantic_1_loss: 0.0093 - semantic_2_loss: 0.0016 - lr: 9.9000e-05
Epoch 3/200
42/42 [==============================] - 23s 549ms/step - loss: 0.0139 - semantic_0_loss: 0.0036 - semantic_1_loss: 0.0088 - semantic_2_loss: 0.0015 - lr: 9.8010e-05
Epoch 4/200
42/42 [==============================] - 24s 552ms/step - loss: 0.0140 - semantic_0_loss: 0.0036 - semantic_1_loss: 0.0089 - semantic_2_loss: 0.0015 - lr: 9.7030e-05
Epoch 5/200
42/42 [==============================] - 23s 546ms/step - loss: 0.0138 - semantic_0_loss: 0.0035 - semantic_1_loss: 0.0088 - semantic_2_loss: 0.0015 - lr: 9.6060e-05
Epoch 6/200
42/42 [==============================] - 23s 547ms/step - loss: 0.0137 - semantic_0_loss: 0.0035 - semantic_1_

...

42/42 [==============================] - 24s 563ms/step - loss: 0.0107 - semantic_0_loss: 0.0029 - semantic_1_loss: 0.0066 - semantic_2_loss: 0.0012 - lr: 2.9049e-05
Epoch 125/200
42/42 [==============================] - 23s 544ms/step - loss: 0.0106 - semantic_0_loss: 0.0029 - semantic_1_loss: 0.0065 - semantic_2_loss: 0.0012 - lr: 2.8758e-05
Epoch 126/200
42/42 [==============================] - 24s 564ms/step - loss: 0.0107 - semantic_0_loss: 0.0029 - semantic_1_loss: 0.0066 - semantic_2_loss: 0.0012 - lr: 2.8471e-05
Epoch 127/200
42/42 [==============================] - 23s 550ms/step - loss: 0.0105 - semantic_0_loss: 0.0029 - semantic_1_loss: 0.0065 - semantic_2_loss: 0.0012 - lr: 2.8186e-05
Epoch 128/200
42/42 [==============================] - 24s 548ms/step - loss: 0.0108 - semantic_0_loss: 0.0029 - semantic_1_loss: 0.0067 - semantic_2_loss: 0.0012 - lr: 2.7904e-05
Epoch 129/200
42/42 [==============================] - 23s 549ms/step - loss: 0.0106 - semantic_0_loss: 0.0029 - s

42/42 [==============================] - 24s 550ms/step - loss: 0.0104 - semantic_0_loss: 0.0028 - semantic_1_loss: 0.0064 - semantic_2_loss: 0.0012 - lr: 2.3522e-05
Epoch 146/200
42/42 [==============================] - 23s 550ms/step - loss: 0.0106 - semantic_0_loss: 0.0029 - semantic_1_loss: 0.0065 - semantic_2_loss: 0.0012 - lr: 2.3286e-05
Epoch 147/200
42/42 [==============================] - 24s 551ms/step - loss: 0.0104 - semantic_0_loss: 0.0029 - semantic_1_loss: 0.0064 - semantic_2_loss: 0.0012 - lr: 2.3054e-05
Epoch 148/200
42/42 [==============================] - 23s 549ms/step - loss: 0.0104 - semantic_0_loss: 0.0029 - semantic_1_loss: 0.0064 - semantic_2_loss: 0.0012 - lr: 2.2823e-05
Epoch 149/200
42/42 [==============================] - 24s 558ms/step - loss: 0.0105 - semantic_0_loss: 0.0029 - semantic_1_loss: 0.0065 - semantic_2_loss: 0.0012 - lr: 2.2595e-05
Epoch 150/200
42/42 [==============================] - 24s 558ms/step - loss: 0.0104 - semantic_0_loss: 0.0028 - s

42/42 [==============================] - 24s 557ms/step - loss: 0.0103 - semantic_0_loss: 0.0028 - semantic_1_loss: 0.0063 - semantic_2_loss: 0.0012 - lr: 1.9239e-05
Epoch 166/200
42/42 [==============================] - 23s 538ms/step - loss: 0.0104 - semantic_0_loss: 0.0029 - semantic_1_loss: 0.0064 - semantic_2_loss: 0.0012 - lr: 1.9046e-05
Epoch 167/200
42/42 [==============================] - 23s 548ms/step - loss: 0.0102 - semantic_0_loss: 0.0028 - semantic_1_loss: 0.0062 - semantic_2_loss: 0.0011 - lr: 1.8856e-05
Epoch 168/200
42/42 [==============================] - 23s 543ms/step - loss: 0.0103 - semantic_0_loss: 0.0028 - semantic_1_loss: 0.0063 - semantic_2_loss: 0.0012 - lr: 1.8667e-05
Epoch 169/200
42/42 [==============================] - 23s 545ms/step - loss: 0.0102 - semantic_0_loss: 0.0028 - semantic_1_loss: 0.0062 - semantic_2_loss: 0.0011 - lr: 1.8480e-05
Epoch 170/200
42/42 [==============================] - 24s 558ms/step - loss: 0.0104 - semantic_0_loss: 0.0029 - s

42/42 [==============================] - 23s 545ms/step - loss: 0.0101 - semantic_0_loss: 0.0028 - semantic_1_loss: 0.0062 - semantic_2_loss: 0.0011 - lr: 1.5578e-05
Epoch 187/200
42/42 [==============================] - 24s 552ms/step - loss: 0.0102 - semantic_0_loss: 0.0028 - semantic_1_loss: 0.0062 - semantic_2_loss: 0.0011 - lr: 1.5422e-05
Epoch 188/200
42/42 [==============================] - 23s 547ms/step - loss: 0.0102 - semantic_0_loss: 0.0028 - semantic_1_loss: 0.0062 - semantic_2_loss: 0.0011 - lr: 1.5268e-05
Epoch 189/200
42/42 [==============================] - 24s 556ms/step - loss: 0.0102 - semantic_0_loss: 0.0028 - semantic_1_loss: 0.0062 - semantic_2_loss: 0.0011 - lr: 1.5115e-05
Epoch 190/200
42/42 [==============================] - 24s 551ms/step - loss: 0.0100 - semantic_0_loss: 0.0028 - semantic_1_loss: 0.0061 - semantic_2_loss: 0.0011 - lr: 1.4964e-05
Epoch 191/200
42/42 [==============================] - 24s 562ms/step - loss: 0.0100 - semantic_0_loss: 0.0028 - s

FileNotFoundError: [Errno 2] No such file or directory: 'images/training/testimages'

In [12]:
import os
print(os.getcwd())
INPUT_PATH="images/test_images/"
MASK_PATH="images/test_gtmasks/"
FILENAMES = [f for f in os.listdir(MASK_PATH)]
print(FILENAMES)


# more detailed mapping of image_mpp parameter
mpps=[1,1.3]

AP_mat=[]
ARI_mat=[]
for CURR_IM_NAME in FILENAMES:
    print(CURR_IM_NAME)
    im0      = io.imread(os.path.join(INPUT_PATH, CURR_IM_NAME.replace("masks","img")))
    mask_true= io.imread(os.path.join(MASK_PATH,  CURR_IM_NAME))

    x = np.expand_dims(im0, axis=-1)
    x = np.expand_dims(x, axis=0)
    y, tile_info = app._tile_input(x)
    AP_arr=[]
    ARI_arr=[]
    for mpp in mpps:
        prd = app._untile_output(app.predict(y, image_mpp=mpp), tile_info)[0,:,:,0]
        AP_arr.append(csi(mask_true, prd))
        ARI_arr.append(adjusted_rand_score(mask_true.flatten(), prd.flatten()))
    AP_mat.append(AP_arr)
    ARI_mat.append(ARI_arr)

print(AP_mat)
print(ARI_mat)

import pandas as pd
df = pd.DataFrame([FILENAMES, [x[0] for x in AP_mat], [x[1] for x in AP_mat]])
print(df.transpose())
df.to_csv('images/training/csi_'+model_name+'.txt', index=False, header=False)

df = pd.DataFrame([FILENAMES, [x[0] for x in ARI_mat], [x[1] for x in ARI_mat]])
print(df.transpose())
df.to_csv('images/training/ari_'+model_name+'.txt', index=False, header=False)

/home/yfong/deeplearning/dense_cell_segmentation
['M872956_JML_Position9_CD3_test_masks.png', 'M872956_JML_Position8_CD3_test_masks.png', 'M872956_JML_Position10_CD3_test_masks.png', 'M872956_JML_Position8_CD4_test_masks.png', 'M926910_CFL_Position13_CD3_test_masks.png', 'M872956_JML_Position8_CD8_test_masks.png', 'M926910_CFL_Position7_CD3_test_masks.png']
M872956_JML_Position9_CD3_test_masks.png


/app/software/DeepCell/0.11.1-foss-2021b-CUDA-11.4.1/lib/python3.9/site-packages/deepcell_toolbox/deep_watershed.py:179: FutureWarning: `selem` is a deprecated argument name for `h_maxima`. It will be removed in version 1.0.Please use `footprint` instead.
  markers = h_maxima(image=maxima,
/app/software/DeepCell/0.11.1-foss-2021b-CUDA-11.4.1/lib/python3.9/site-packages/deepcell_toolbox/deep_watershed.py:179: FutureWarning: `selem` is a deprecated argument name for `h_maxima`. It will be removed in version 1.0.Please use `footprint` instead.
  markers = h_maxima(image=maxima,


M872956_JML_Position8_CD3_test_masks.png


/app/software/DeepCell/0.11.1-foss-2021b-CUDA-11.4.1/lib/python3.9/site-packages/deepcell_toolbox/deep_watershed.py:179: FutureWarning: `selem` is a deprecated argument name for `h_maxima`. It will be removed in version 1.0.Please use `footprint` instead.
  markers = h_maxima(image=maxima,
/app/software/DeepCell/0.11.1-foss-2021b-CUDA-11.4.1/lib/python3.9/site-packages/deepcell_toolbox/deep_watershed.py:179: FutureWarning: `selem` is a deprecated argument name for `h_maxima`. It will be removed in version 1.0.Please use `footprint` instead.
  markers = h_maxima(image=maxima,


M872956_JML_Position10_CD3_test_masks.png


...

M872956_JML_Position8_CD8_test_masks.png


/app/software/DeepCell/0.11.1-foss-2021b-CUDA-11.4.1/lib/python3.9/site-packages/deepcell_toolbox/deep_watershed.py:179: FutureWarning: `selem` is a deprecated argument name for `h_maxima`. It will be removed in version 1.0.Please use `footprint` instead.
  markers = h_maxima(image=maxima,
/app/software/DeepCell/0.11.1-foss-2021b-CUDA-11.4.1/lib/python3.9/site-packages/deepcell_toolbox/deep_watershed.py:179: FutureWarning: `selem` is a deprecated argument name for `h_maxima`. It will be removed in version 1.0.Please use `footprint` instead.
  markers = h_maxima(image=maxima,


M926910_CFL_Position7_CD3_test_masks.png


/app/software/DeepCell/0.11.1-foss-2021b-CUDA-11.4.1/lib/python3.9/site-packages/deepcell_toolbox/deep_watershed.py:179: FutureWarning: `selem` is a deprecated argument name for `h_maxima`. It will be removed in version 1.0.Please use `footprint` instead.
  markers = h_maxima(image=maxima,
/app/software/DeepCell/0.11.1-foss-2021b-CUDA-11.4.1/lib/python3.9/site-packages/deepcell_toolbox/deep_watershed.py:179: FutureWarning: `selem` is a deprecated argument name for `h_maxima`. It will be removed in version 1.0.Please use `footprint` instead.
  markers = h_maxima(image=maxima,


[[0.34763948497854075, 0.4405737704918033], [0.4597069597069597, 0.6043956043956044], [0.3596330275229358, 0.46371681415929206], [0.5792349726775956, 0.5714285714285714], [0.30133333333333334, 0.3713592233009709], [0.48863636363636365, 0.48756218905472637], [0.3959731543624161, 0.42484662576687116]]
[[0.6792525921434196, 0.7008687737987954], [0.6312479463058475, 0.6661010014792792], [0.6452686501958117, 0.6459803567964051], [0.7351109824507992, 0.7340661472858063], [0.6644093678134864, 0.6337896724337856], [0.7491257244978152, 0.749604177689936], [0.5589633015274665, 0.5531025309833009]]
                                           0         1         2
0   M872956_JML_Position9_CD3_test_masks.png  0.347639  0.440574
1   M872956_JML_Position8_CD3_test_masks.png  0.459707  0.604396
2  M872956_JML_Position10_CD3_test_masks.png  0.359633  0.463717
3   M872956_JML_Position8_CD4_test_masks.png  0.579235  0.571429
4  M926910_CFL_Position13_CD3_test_masks.png  0.301333  0.371359
5   M872956_JML